In [35]:
from __future__ import print_function
import sys
import os
import argparse
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from data.ant_test import ANT_ROOT, ANT_CLASSES as labelmap
from PIL import Image
from data.ant_test import ANTAnnotationTransform, ANTDetection, ANT_CLASSES
from data import BaseTransform
import torch.utils.data as data
from ssd import build_ssd
import cv2


In [36]:
num_classes = len(ANT_CLASSES) + 1 # +1 background
net = build_ssd('test', 300, num_classes) # initialize SSD
net.load_state_dict(torch.load('weights/ssd300_ant_2900.pth'))
net.eval()

D:\ant_dec\ssd.pytorch\ssd.py:35: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
D:\ant_dec\ssd.pytorch\layers\modules\l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (17): Conv2d(256, 

**`opencv`单张图片做预测**

In [68]:
img = cv2.imread('./ANT/Testimages/11381045_b352a47d8c.jpg')
transform = BaseTransform(net.size, (104, 117, 123))

x = torch.from_numpy(transform(img)[0]).permute(2, 0, 1)
x = Variable(x.unsqueeze(0))

y = net(x)      # forward pass
detections = y.data
scale = torch.Tensor([img.shape[1], img.shape[0],img.shape[1], img.shape[0]])
for i in range(detections.size(1)):
    j = 0
    while detections[0, i, j, 0] >= 0.2:
        score = detections[0, i, j, 0]
        label_name = labelmap[i-1]
        pt = (detections[0, i, j, 1:]*scale).cpu().numpy()
        coords = (pt[0], pt[1], pt[2], pt[3])


        cv2.rectangle(img, (coords[0], coords[1]),(coords[2], coords[3]), (0, 255, 0), 1)
        cv2.putText(img, '{}:{:.3f}'.format(label_name, score.item()), (coords[0], coords[1]), \
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
        j += 1
        
cv2.imshow('image', img)
cv2.waitKey(1) & 0xFF == ord('q')
cv2.destroyAllWindows()


**`opencv`读取视频进行目标检测**

In [69]:
cap = cv2.VideoCapture(r"C://Users/gtw/Desktop/video/Two Giant Killer Hornet Colonies Fight to the Death.mp4")
num_classes = len(ANT_CLASSES) + 1 # +1 background
net = build_ssd('test', 300, num_classes) # initialize SSD
net.load_state_dict(torch.load('weights/ssd300_ant_2900.pth'))
net.eval()

while cap.isOpened():
    ok, frame = cap.read()  # 读取一帧数据
    # ipdb.set_trace()
    if not ok:
        break
    transform = BaseTransform(net.size, (104, 117, 123))

#     image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    x = torch.from_numpy(transform(frame)[0]).permute(2, 0, 1)
    x = Variable(x.unsqueeze(0))

    y = net(x)      # forward pass
    detections = y.data
    scale = torch.Tensor([frame.shape[1], frame.shape[0],frame.shape[1], frame.shape[0]])
    for i in range(detections.size(1)):
        j = 0
        while detections[0, i, j, 0] >= 0.3:
            score = detections[0, i, j, 0]
            label_name = labelmap[i-1]
            pt = (detections[0, i, j, 1:]*scale).cpu().numpy()
            coords = (pt[0], pt[1], pt[2], pt[3])


            cv2.rectangle(frame, (coords[0], coords[1]),(coords[2], coords[3]), (0, 255, 0), 1)
            cv2.putText(frame, '{}:{:.3f}'.format(label_name, score.item()), (coords[0], coords[1]), \
                        cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
            j += 1
            
    cv2.imshow('images', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

D:\ant_dec\ssd.pytorch\ssd.py:35: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
D:\ant_dec\ssd.pytorch\layers\modules\l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)
